In [6]:
%run imports.ipynb

imports loaded.



In [15]:
print('> file management func:\n')

> file management func:



In [25]:
def get_content(directory):
    content = []
    for file in os.scandir(directory):
        if file.is_dir():
            content.append(file.path + '/')
        if file.is_file():
            content.append(file.path)
    return content
    
print('    get_content(directory)')

    get_content(directory)


In [8]:
def get_last_update(file_path):

    last_modified_timestamp = os.path.getmtime(file_path)
    last_modified_datetime = datetime.fromtimestamp(last_modified_timestamp)
    return last_modified_datetime.strftime("%d-%m-%Y")

print('    get_last_update(file_path)')

    get_last_update(file_path)


In [18]:
print('\n> fighters_stats_scrape func:\n')


> fighters_stats_scrape func:



In [53]:
def get_xml(url, headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}):
    
    try:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'xml')
    except:
        #print('Error on: ' + url) 
        return None
    
    return soup

print("    get_xml(url, headers = {'User-Agent':'Moz...}) # Return web page data in .xml format.")

    get_xml(url, headers = {'User-Agent':'Moz...}) # Return web page data in .xml format.


In [97]:
def get_links(url):
    
    link_items =  get_xml(url=url).find_all('a',{'class':'b-link b-link_style_black'})
    links = []
    for item in link_items:
        links.append(item['href'])
    
    return links
   
print('    get_links(url)')

    get_links(url)


In [111]:
def get_fighter_data(url):
    
    xml =  get_xml(url=url)
    data = {}
    data['name'] = xml.find('h2').find('span',{'class':'b-content__title-highlight'}).text.strip()
    
    record_string = xml.find('h2').find('span',{'class':'b-content__title-record'}).text.strip()
    data['wins'] = int(record_string[record_string.find(':')+1:record_string.find('-')])
    data['losses'] = int(record_string.split('-')[1])
    data['draws'] = record_string.split('-')[2]
    
    
    stat_boxes = get_xml(url=url).find_all('ul')[0:3]
    for item in stat_boxes:
        for sub_item in item.contents:
            text = sub_item.text.strip().replace(" ", "").replace("\n", "")
            if len(text) > 0:
                key = text[:text.find(':')]
                value = text[text.find(':')+1:]
                data[key] = value
    
    data['last_update'] = None
    data['last_fight'] = None
    return data

print('    get_fighter_data(url) # Retrive data as dictionery') 

    get_fighter_data(url) # Retrive data as dictionery


In [110]:
def get_fighter_data(url):
    
    xml =  get_xml(url=url)
    data = []
    data.append(xml.find('h2').find('span',{'class':'b-content__title-highlight'}).text.strip()) #name
    
    record_string = xml.find('h2').find('span',{'class':'b-content__title-record'}).text.strip()
    data.append(int(record_string[record_string.find(':')+1:record_string.find('-')])) #wins
    data.append(int(record_string.split('-')[1])) #losses
    data.append(record_string.split('-')[2]) #draws
    
    stat_boxes = get_xml(url=url).find_all('ul')[0:3]
    for item in stat_boxes:
        for sub_item in item.contents:
            text = sub_item.text.strip().replace(" ", "").replace("\n", "")
            if len(text) > 0:
                data.append(text[text.find(':')+1:])
    
    data.append(None) #last_update
    data.append(None) #last_match
    return np.array(data)

print('    get_fighter_data(url) # Retrive data as list') 

    get_fighter_data(url) # Retrive data as list


In [46]:
print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')

>>>>>


In [13]:
def get_current_date():
    return datetime.date.today().strftime('%d-%m-%Y')
    
print('    get_current_date()')    

    get_current_date()


In [6]:
def get_row(soup):
    
    '''get soup & return np.array of all relevant row details'''
    
    row = get_name(soup)
    row = np.append( row,get_record(soup) )
    row = np.append( row,get_stats(soup) )
    
    return row
print('    get_row(soup)')

    get_row(soup)


In [5]:
def get_record(soup):
    
    '''get soup & return np.array of wins,draws,losses'''
    
    h2 = soup.find('h2',{'class':'b-content__title'})

    record = h2 .contents[3].contents[0].strip()
    record = record[record.find(':')+2:len(record)]
    
    wins = record[:record.find('-')]
    
    losses = record[record.find('-')+1:record.find('-')+record.find('-')]
    
    if record.find(' ') == -1:
        string_end = len(record)
    else:
        string_end = record.find(' ')
    
    draws = record[record.find('-')+1+record.find('-'):string_end]
    
    return np.array([wins,losses,draws])
print('    get_record(soup)')

    get_record(soup)


In [7]:
def get_name(soup):
    
    '''get soup & return fighter's name'''
    
    h2 = soup.find('h2',{'class':'b-content__title'})
    name = h2.contents[1].contents[0].strip()
    
    return np.array([name])
print('    get_name(soup)')

    get_name(soup)


In [8]:
def get_stats(soup):
    
    '''get html data from page & return row with only the relevant values'''
    
    data = get_html_data(soup)
    
    row = []

    for i in range(len(data)):
        for j in range(len(data[i])):
            row.append( data[i][j].contents[2].strip() )
        
    return np.array(row)     
print('    get_stats(soup)')

    get_stats(soup)


In [9]:
def get_columns(soup):
    
    '''return list of columns'''
    
    data = get_html_data(soup)
    
    columns = ['name','wins','losses','draws']
    
    for i in range(len(data)):
        for j in range(len(data[i])):
            columns.append( data[i][j].contents[1].contents[0].strip() )
     
    return columns
print('    get_columns(soup)')

    get_columns(soup)


In [27]:
print('\n> fights_stats_web_scrape func:\n')


> fights_stats_web_scrape func:



In [32]:
def get_event_data(soup):
    #return soup.find_all('th')[0:3] # column names
    return soup.find('p',{'class':'b-fight-details__table-text'}) #data

print('    get_event_data(soup)')

    get_event_data(soup)


In [ ]:

def extract_fight_data(event_url):
    # Send a GET request to the event page
    response = requests.get(event_url)
    
    # Create a BeautifulSoup object from the response content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all the rows in the "FIGHTS" table
    table = soup.find('table', {'class': 'b-fight-details__table'})
    rows = table.find_all('tr', {'class': 'b-fight-details__table-row'})
    
    # Loop through each row and extract the relevant data
    fight_data = []
    for row in rows:
        # Find the columns in the row
        cols = row.find_all('td')
        
        # Extract the data from each column
        fighter1 = cols[1].find('a').text.strip()
        fighter2 = cols[3].find('a').text.strip()
        weight_class = cols[5].text.strip()
        method = cols[6].text.strip()
        round = cols[7].text.strip()
        time = cols[8].text.strip()
        
        # Add the data to the fight_data list
        fight_data.append({
            'fighter1': fighter1,
            'fighter2': fighter2,
            'weight_class': weight_class,
            'method': method,
            'round': round,
            'time': time
        })
    
    return fight_data

print('    extract_fight_data(event_url)')


In [28]:
def get_fight_columns(soup):
    
    column_items = soup.find_all('th',{'class':'b-fight-details__table-col'})
    columns = []
    for item in column_items:
        columns.append(item.text.strip())
          
    return columns
print('    get_fight_columns(soup)')

    get_columns(soup)


In [16]:
def get_fight_list(soup):
    
    fights_list = soup.find_all('tr',{'class':'b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click'})
    return fights_list

    links = fights_list.find('a')
    return links

print('    get_fight_list(soup)')

    get_fight_list(soup)


In [10]:
def get_event_date(soup):
    
    '''return event date'''
    
    item = soup.find('li',{'class':'b-list__box-list-item'})
    string = item.get_text().strip()
    start = len('DATE:')
    date = string[start:].strip()
    
    return datetime.strptime(date, "%B %d, %Y").strftime("%d-%m-%Y")

print('    get_event_date(soup)')

    get_event_date(soup)


In [12]:
def get_later_date(date1, date2):

    date1_obj = datetime.strptime(date1, "%d-%m-%Y")
    date2_obj = datetime.strptime(date2, "%d-%m-%Y")

    if date1_obj > date2_obj:
        return date1
    else:
        return date2
    
print('    get_later_date(date1, date2)')    

    get_later_date(date1, date2)
